# Imports

In [9]:
import bruce, numpy as np, matplotlib.pyplot as plt, os
from astropy.table import Table, Column
from astropy.stats import sigma_clip
from astropy.time import Time , TimeDelta
from astropy import units as u

# Load the tics

Data should have the form of (ascii or CSV, your choice). t_zero is in full BJD, width is in days, depth is in normalised flux. If you ue SPOCFIT, if you fit a single transit you will see 3 reported values on the bottom row which are these values for each event (t_zero, width, depth).

tic_id,	t_zero_1,	width_1,	depth_1,	t_zero_2,	width_2,	depth_2,

In [4]:
duos = Table.read('duos.csv')
duos['tic_id'] = duos['TIC ID']
print(duos)

  TIC ID  Linked TIC ID ...   tic_id 
--------- ------------- ... ---------
  7145074       7145074 ...   7145074
 20904104      20904104 ...  20904104
 22317640      22317640 ...  22317640
 32179255      32179255 ...  32179255
 42428568      42428568 ...  42428568
 52195587      52195587 ...  52195587
      ...           ... ...       ...
 39904176      39904176 ...  39904176
116261487     116261487 ... 116261487
  5267885       5267885 ...   5267885
275267824     275267824 ... 275267824
286969201     286969201 ... 286969201
103095888     103095888 ... 103095888
Length = 86 rows


# Now the main worker function

This function does the following:
-  Load the latest TESS data
-  Flatten the lightcurve
-  Fit the events
-  Calcualte the aliases
-  Plot the permissable aliases
-  Create a report

In [12]:
for i in range(len(duos))[:]:   
    # Create the output dir (we'll use this as cache for the data too)
    output_dir = os.getcwd() + '/{:}'.format(duos['tic_id'][i])
    os.system('mkdir -p {:}'.format(output_dir))
    if os.path.isfile(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i])) : continue

    # Now load the TESS data (SPOC, QLP)
    # We are not making our own here like TESSTPF, not yet anyway...
    # for data_type
    #   single_product -> all sectors together
    #   per_sector -> list of per-sector lightcurves
    #   northern_duos -> YEARS 2 and 4, then a list of other sectors 
    #   southern -> YEARS 1 and 3, then a list of other sectors (NOT IMPLEMENTED YET) 
    t, data,data_labels, base_dir =  bruce.ambiguous_period.download_tess_data(duos['tic_id'][i], 
                                                              max_sector=None, 
                                                                   use_ffi=True, 
                                                                   download_dir=None, 
                                                                   bin_length=0.5/24)
    
    # Now flatten the data
    for j, k in zip(data, data_labels):
        # Flatten the data by SG filter, we need an odd kernel legth based on cadence
        j.flatten_data_old(window_width=3, sigmaclip=3, dx_lim=0.1)


#         for seg in bruce.data.find_nights_from_data(j.time, dx_lim=0.2):
#             j.w = np.ones(j.time.shape[0])*np.median(j.flux)

        # Optinally save the data
        j.write_data(output_dir + '/' +'TESS_DATA_{:}.txt'.format(k))
        fig, ax = j.plot_segments(dx_lim=0.5)
        fig.savefig(output_dir + '/' + 'TESS_DATA_{:}.png'.format(k))
        plt.close(fig)

    # # Now re-order_datasets based on epochs given
    # We will unpack now, (data with transits, and data without)
    # We may need to change this for it to work properly (Sam is working on it)
    # Its worth noting we can incorparate ground based data here too
    # data_from_ground = bruce.ambiguous_period.mono_event.photometry_time_series(time, flux, flux_err, w = norm_model)
    # Then this can go in data_other_sectors
    data, data_labels = bruce.ambiguous_period.group_data_by_epochs(data, data_labels, duos['t_zero_1'][i], duos['t_zero_2'][i])
    data, data_other_sectors = data[0], data[1:]



    ############################
    # FIT EVENT 1
    ############################
    # Mask data and create the mono_event object
    nmask = 3
    mask1 = (data.time > (duos['t_zero_1'][i] - nmask*duos['width_1'][i])) &  (data.time < (duos['t_zero_1'][i] + nmask*duos['width_1'][i]))
    data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], data.flux_err[mask1], w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    m1 = bruce.ambiguous_period.mono_event(duos['t_zero_1'][i], duos['width_1'][i], duos['depth_1'][i], data_event_1, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = None)
    
    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_1 = m1.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)


    ############################
    # FIT EVENT 2
    ############################
    # Mask data and create the mono_event object
    mask2 = (data.time > (duos['t_zero_2'][i] - nmask*duos['width_2'][i])) &  (data.time < (duos['t_zero_2'][i] + nmask*duos['width_2'][i]))
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], data.flux_err[mask2], w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    m2 = bruce.ambiguous_period.mono_event(duos['t_zero_2'][i], duos['width_2'][i], duos['depth_2'][i], data_event_2, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_2 = m2.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)

    # We are going to make a nice plot of the two events with their models
    fig, ax = plt.subplots(1,2, gridspec_kw={'hspace' : 0, 'wspace' : 0}, figsize = (6.4, 3.8))
    ax[0].errorbar(return_data_1[0], return_data_1[1], yerr=return_data_1[2], fmt='k.', alpha = 0.1)
    ax[0].plot(return_data_1[3], return_data_1[4], c='orange')
    ax[1].errorbar(return_data_2[0], return_data_2[1], yerr=return_data_2[2], fmt='k.', alpha = 0.1)
    ax[1].plot(return_data_2[3], return_data_2[4], c='orange')
    ax[1].set(yticks=[])
    ylim1 = ax[0].get_ylim()
    ylim2 = ax[1].get_ylim()
    ylim = [min(ylim1[0],ylim2[0]), max(ylim1[1], ylim2[1])]
    ax[0].set_ylim(ylim)
    ax[1].set_ylim(ylim)
    fig.supxlabel('Time from Transit [d]', fontsize=18, x=0.55, y = -0.005)
    fig.supylabel('Flux', fontsize=18)
    fig.suptitle(m2.name, y=0.95, x=0.55, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3', alpha=1.0), ha='center', fontsize=18)
    plt.subplots_adjust(right=0.99, top=0.99, bottom=0.13)
    fig.savefig(output_dir + '/' + 'TIC-{:}_BOTH_EVENTS.png'.format(duos['tic_id'][i]))
    plt.close(fig)

    ########################################################
    # CREATE THE AMBIGUOUS PERIOD OBJECT
    ########################################################
    p = bruce.ambiguous_period.ambiguous_period(data, events=[m1,m2], name='TIC-{:}'.format(duos['tic_id'][i]),
                        median_bin_size = 2,convolve_bin_size = 2)

    # Now mask and filter 
    p.mask_and_filter_events()

    # Calculate aliases
    # Do not use nsolutions_events here (that is superceeded later)
    nsolutions_events = p.calcualte_aliases(dx_lim=0.03, min_period=15)

    # Now calcualte whether we saw a transit by comparing the model to a flat line
    delta_L_data = p.calcualte_data_delta_L(data)
    #delta_L_data = delta_L_data + 100 # THIS FUDGE IS OFTEN NEEDED
    for j in range(delta_L_data.shape[0]):
        print(j, delta_L_data[j])
    delta_L_data_from_other_sectors_or_others = [p.calcualte_data_delta_L(j) for j in data_other_sectors]
    p.delta_L = np.array([delta_L_data, *delta_L_data_from_other_sectors_or_others])


    ########################################################
    # Plot the aliases
    ########################################################
    fig, ax  = p.plot_aliases(phot_data=data_other_sectors, phot_data_labels=data_labels)
    fig.savefig(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i]), dpi=300)
    plt.close(fig)
    
    ########################################################
    # Now report the aliases
    ########################################################
    aliases = (p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    periods = (p.max_period/p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    alaises = Table()
    alaises.add_column(Column(duos['tic_id'][i]*np.ones(len(aliases), dtype=int), name='tic_id'))
    alaises.add_column(Column(m1.de_get_epoch()*np.ones(len(aliases)), name='t_zero_1'))
    alaises.add_column(Column(m1.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_1'))
    alaises.add_column(Column(m1.de_get_k()*np.ones(len(aliases)), name='k_1'))
    alaises.add_column(Column(m1.de_get_b()*np.ones(len(aliases)), name='b_1'))
    alaises.add_column(Column(m1.de_transit_width()*np.ones(len(aliases)), name='width_1'))
    alaises.add_column(Column(m2.de_get_epoch()*np.ones(len(aliases)), name='t_zero_2'))
    alaises.add_column(Column(m2.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_2'))
    alaises.add_column(Column(m2.de_get_k()*np.ones(len(aliases)), name='k_2'))
    alaises.add_column(Column(m2.de_get_b()*np.ones(len(aliases)), name='b_2'))
    alaises.add_column(Column(m2.de_transit_width()*np.ones(len(aliases)), name='width_2'))
    alaises.add_column(Column(aliases, name='alias'))
    alaises.add_column(Column(periods, name='period'))
    alaises.write(output_dir + '/' + 'TIC-{:}_ALIASES.fits'.format(duos['tic_id'][i]), overwrite=True)

    
    ########################################################
    # Now plan the aliases
    ########################################################
    transit_events = p.transit_plan(start=Time.now(), end = Time.now()+TimeDelta(30, format='jd'), resolution = 1*u.minute,
                    tic_id=duos['tic_id'][i], observatory='Paranal',
                    sun_max_alt=-15, target_min_alt=30, moon_min_seperation=20,
                    min_time_in_transit=None, min_frac_in_transit=None)
    transit_events.write(output_dir + '/' + 'TIC-{:}_ALIASES_WINDOWS_PARANAL.fits'.format(duos['tic_id'][i]), overwrite=True)
        


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpa69cwvqv

Querying TIC 118339710 from MAST...



✅ Download summary:
Sector ...
------ ...
     8 ...
    34 ...
    35 ...
    61 ...
    88 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpa69cwvqv
This directory will be deleted when the program exits.
W :  143
W :  144


W :  144
W :  144
W :  144
W :  30
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  21
W :  144
W :  144
Initial Chi-Sqaured : 448.96 [red 4.08]
Fitted parameters for TIC-118339710:
t_zero : 2458525.646933233
radius_1 : 0.051686085237850246
k : 0.06518645518263097
b : 0.7330604602222146
Final Chi-Sqaured : 101.77 [red 0.93]
Initial Chi-Sqaured : 217.40 [red 2.03]
Fitted parameters for TIC-118339710:
t_zero : 2459249.5488770576
radius_1 : 0.05110643084762126
k : 0.06323296530024211
b : 0.7223750546297399
Final Chi-Sqaured : 113.88 [red 1.06]
1 49 1 48.26012958831464 723.9019438247196 15 15.081290496348325
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 -1419.9609810075629
36 -1590.3796357144486
37 -1898.5294065343573
38 -2040.8257176536065
39 -1427.0664420187275
40 -1265.7113008303

100%|███████████████████████████████████████████| 48/48 [00:06<00:00,  7.38it/s]


No transists visible

📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpx65xvwt5

Querying TIC 121077168 from MAST...


...

No transists visible

📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpon35fubw

Querying TIC 103095888 from MAST...



✅ Download summary:
Sector SPOC FFI QLP LC TESS-SPOC LC SPOC LC     Source                                                                File                                                            
------ -------- ------ ------------ ------- ------------- ----------------------------------------------------------------------------------------------------------------------------
    19        X     --           --      -- FFI custom LC          /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpon35fubw/tess-s0019-2-2_61.135848_74.538163_10x10_astrocut.fits
    25       --     --           --       X       SPOC LC         /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpon35fubw/tess2020133194932-s0025-0000000103095888-0182-s_lc.fits
    26       --     --           --       X       SPOC LC         /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpon35fubw/tess2020160202036-s0026-0000000103095888-0188-s_lc.fits
    52       --     --            X      --  TESS-SPOC LC /var/f

W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  28
W :  10
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  12
W :  144
W :  144
W :  13
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  70
W :  36
W :  62
W :  144
W :  20
W :  28
W :  144
Initial Chi-Sqaured : 128.70 [red 2.63]
Fitted parameters for TIC-103095888:
t_zero : 2459015.829762211
radius_1 : 0.02483057519095777
k : 0.037841026576112255
b : 0.3579943894022634
Final Chi-Sqaured : 90.72 [red 1.85]
Initial Chi-Sqaured : 283.04 [red 7.86]
Fitted parameters for TIC-103095888:
t_zero : 2459746.009518684
radius_1 : 0.03982185702670529
k : 0.03672267696782436
b : 0.7848241142148323
Final Chi-Sqaured : 241.32 [red 6.70]
1 49 1 48.678650431521234 730.1797564728186 15 15.212078259850387
0 0.0
1 0.0
2 0.0
3 -0.019197678727411453
4 0.0
5 0.0
6 0.0
7 -0.01919767872741613
8 0.0
9 0.0
10 -0.12491407020858801
11 -0.019197678727414964
12 0.0
13 0.0
14 -0.13490269677627595
15 -0.019197678727414527
16 0.0
17 0.0
18 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:12<00:00,  3.83it/s]


No transists visible


# Now lok for solved systems

In [13]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p solved_systems; rm solved_systems/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==1:
        os.system('cp {:}/TIC-{:}_ALIASES.png solved_systems'.format(tic_id,tic_id))
        
        print('{:}, {:}, {:}'.format(tic_id,  t['t_zero_1'][0], t['period'][0]))
        

rm: cannot remove 'solved_systems/*': No such file or directory


224279805, 2458369.727329248, 22.984009901774698
52195587, 2458350.2770604594, 47.33320196554996
356158613, 2458713.0097145545, 21.388638748029503
257024338, 2458922.4813573235, 81.43377512902953
396720998, 2458399.237195914, 46.35443516011583
229476204, 2458743.522001057, 648.058725417126


In [ ]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p solved_systems; rm solved_systems/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==1:
        os.system('cp {:}/TIC-{:}_ALIASES.png solved_systems'.format(tic_id,tic_id))
        
        print('{:}, {:}, {:}'.format(tic_id,  t['t_zero_1'][0], t['period'][0]))
        

# Now make predictions about when they will transit

In [ ]:
from astropy.table import vstack
vstack([Table.read(i) for i in files])